In [2]:
from tensorflow import keras
import tensorflow as tf
import cv2
import os



In [22]:
from keras.preprocessing.image import ImageDataGenerator


data_dir = "/home/ayush/Projects/Facial-Recognition-on-Blink-Doorbell/Data/Name(to_train)"
# rescale=1./255, 
data_generator = ImageDataGenerator(validation_split=0.2)


train_data = data_generator.flow_from_directory(
    directory=data_dir,
    target_size=(100, 100),
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    subset="training"
)

val_data = data_generator.flow_from_directory(
    directory=data_dir,
    target_size=(100, 100),
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    subset="validation"
)



Found 153 images belonging to 1 classes.
Found 38 images belonging to 1 classes.


ValueError: too many values to unpack (expected 2)

In [4]:
from keras.applications import vgg16

model = vgg16.VGG16(weights = 'imagenet',
                    include_top=False,
                    input_shape=(100,100,3))





2023-03-15 18:05:31.576868: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-15 18:05:31.577101: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-15 18:05:31.577192: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-15 18:05:31.577692: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-15 18:05:31.577725: I tensorflow/core/co

In [5]:
for layer in model.layers:
    layer.trainable = False
    

In [6]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0     

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import BatchNormalization
from keras.models import Model

def layer_adder(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation='relu')(top_model)
    top_model = Dense(512, activation='relu')(top_model)
    top_model = Dense(num_classes, activation='sigmoid')(top_model)
    return top_model


In [17]:


num_classes = 1

FC_Head = layer_adder(model, num_classes)
model = Model(inputs = model.input, outputs = FC_Head)


In [18]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0   

In [36]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = "/home/ayush/Projects/Facial-Recognition-on-Blink-Doorbell/Data/Name(to_train)/Swapnil/train"
validaton_data_dir = "/home/ayush/Projects/Facial-Recognition-on-Blink-Doorbell/Data/Name(to_train)/Swapnil/validation"

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(100,100),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validaton_data_dir,
    target_size=(100,100),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 161 images belonging to 1 classes.
Found 30 images belonging to 1 classes.


In [40]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("detector_weights.h15",
                             monitor="val_loss",
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor="val_loss",
                          min_delta= 0,
                          patience = 3,
                          verbose = 1,
                          restore_best_weights=True)

callbacks = [earlystop, checkpoint]

model.compile(loss = "categorical_crossentropy",
              optimizer = "Adam",
              metrics = ['accuracy'])

nb_train_samples = 161
nb_validation_samples = 30

epochs = 1
batch_size = 16

history = model.fit_generator(train_generator,
                              epochs=epochs,
                              callbacks=callbacks,
                              validation_data= validation_generator)

/tmp/ipykernel_274/140307228.py:27: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


6/6 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0870
Epoch 1: val_loss improved from inf to 0.00000, saving model to detector_weights.h15


INFO:tensorflow:Assets written to: detector_weights.h15/assets


INFO:tensorflow:Assets written to: detector_weights.h15/assets


6/6 [==============================] - 9s 1s/step - loss: 0.0000e+00 - accuracy: 0.0870 - val_loss: 0.0000e+00 - val_accuracy: 0.1000


In [41]:
from keras.models import load_model

classifier = load_model('detector_weights.h15')

In [ ]:
from funcs import extract_images,resize_images

